In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB


In [2]:
#Giving File Location
train_file = 'data/labeledTrainData.tsv'
unlabeled_train_file = 'data/unlabeledTrainData.tsv'
test_file = 'data/testData.tsv'
output_file = 'data/submit_200_4.csv'

In [3]:
#Loading Data
train = pd.read_csv( train_file, header = 0, delimiter = "\t", quoting = 3 )
test = pd.read_csv( test_file, header = 0, delimiter = "\t", quoting = 3 )
unlabeled_train = pd.read_csv( unlabeled_train_file, header = 0, delimiter= "\t", quoting = 3 )

In [6]:
print("Parsing train reviews...")
clean_train_reviews = [] 
for review in train['review']:
    clean_train_reviews.append( " ".join( KaggleWord2VecUtility.review_to_wordlist( review )))

Parsing train reviews...


In [7]:
print("Parsing unlabeled_train reviews...") 
unlabeled_clean_train_reviews = []
for review in unlabeled_train['review']:
    unlabeled_clean_train_reviews.append( " ".join( KaggleWord2VecUtility.review_to_wordlist( review )))

Parsing unlabeled_train reviews...


In [8]:
print("Parsing test reviews...")
clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append( " ".join( KaggleWord2VecUtility.review_to_wordlist( review )))

Parsing test reviews...


In [ ]:
print("Vectorizing...")

vectorizer = TfidfVectorizer( min_df=2, max_df=0.95, max_features = 200000, ngram_range = ( 1, 4 ),
                              sublinear_tf = True )

vectorizer = vectorizer.fit(clean_train_reviews + unlabeled_clean_train_reviews)
train_data_features = vectorizer.transform( clean_train_reviews )
test_data_features = vectorizer.transform( clean_test_reviews )

Vectorizing...


In [23]:
print("Reducing dimension...")

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
fselect = SelectKBest(chi2 , k=70000)
train_data_features = fselect.fit_transform(train_data_features, train["sentiment"])
test_data_features = fselect.transform(test_data_features)

Reducing dimension...


In [24]:
print("Training...")

model1 = MultinomialNB(alpha=0.0005)
model1.fit( train_data_features, train["sentiment"] )

model2 = LogisticRegression(penalty="l1")
model2.fit( train_data_features, train["sentiment"] )

p1 = model1.predict_proba( test_data_features )[:,1]
p2 = model2.predict_proba( test_data_features )[:,1]

print("Writing results...")

output = pd.DataFrame( data = { "id": test["id"], "sentiment": (p1 + p2)/2 } )
output.to_csv( 'output_file.csv', index = False, quoting = 3 ) 

Training...
Writing results...


![](./data/naieve_bayes.png) 

In [5]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')



class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review,"lxml").get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]

        b=[]
        stemmer = english_stemmer #PorterStemmer()
        for word in words:
            b.append(stemmer.stem(word))

        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences 

In [ ]:
# print("Training...")

# model1 = MultinomialNB(alpha=0.0005)
# model1.fit( train_data_features, train["sentiment"] )

# model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
# model2.fit( train_data_features, train["sentiment"] )

# p1 = model1.predict_proba( test_data_features )[:,1]
# p2 = model2.predict_proba( test_data_features )[:,1]

# print("Writing results...")

# output = pd.DataFrame( data = { "id": test["id"], "sentiment": .2*p1 + 1.*p2 } )
# output.to_csv( 'output_file.csv', index = False, quoting = 3 ) 